# FastSLAM

FastSLAM também conhecido como Rao-blackwellized Filtro de párticula, usa partícula de filtro para estimar o caminho do robô. Para cada uma dessas partícula o erro individual do map são condicionalmente independentes. Portanto, o problema de mapeamento pode ser fatorado em muitos problemas separados, um para cada característica no mapa. FastSLAM estima essas característica locais pelo EKFs (Extended Kalman Filter), mas usando um EKF de baixa dimensão separado para cada recurso individual.

## Fatorando o SLAM Posterior

O conceito matemático principal, refere-se que todo o posterior SLAM, pode ser reescrito na forma fatorada.

$ p(s^{t}, \Theta | z^{t}, u^{t}, n^{t}) = p(s^{t} | z^{t}, u^{t}, n^{t}) \cdot p(\Theta | s^{t}, z^{t}, u^{t}, n^{t})$

$ p(s^{t}, \Theta | z^{t}, u^{t}, n^{t}) = p(s^{t} | z^{t}, u^{t}, n^{t}) \cdot \prod_{i=1}^{N} p(\theta_n | s^{t}, z^{t}, u^{t}, n^{t})$

Onde,

N = Número total de landmarkers

M = Número total de partículas

$s^t$ = lista com as posições

$s_{t}$ = Posição do Robô

$ \theta_n$ = Posição do Enesímo landmarker

$ \Theta $= map, lista com todas as posições dos landmarkers

$ n_t $ = indice do landmarker

$z_{1:t}$ = Sensores

$u_{0:t-1}$ = Controle

Para cada partícula, a distribuição conjunta da pose e do mapa, dadas as observações e os movimentos, é fatorizada, baseado no Rao-Blackwellization, e reescrito como o produto da probabilidade da pose dadas as observações e os movimentos (path posterior) e a probabilidade do mapa dada a posição e as observações (map posterior).

## Pseudocode


1.  &nbsp;$EKF(μ_{t−1}, Σ_{t−1}, u_t, z_t):$

2.  &nbsp;$  \bar{\mu}_t = g(u_t, \mu_{t-1} )$

3.  &nbsp;$ \bar{\Sigma}_t = G_t \Sigma_{t−1} G^T_t + R_t$

4.  &nbsp;$  K_t = \bar{\Sigma}_t H^T_t (H_t \bar{\Sigma_t} H^T_t + Q_t)^{−1}$

5.  &nbsp;$ \mu_t = \bar{\mu}_t + K_t(z_t − h(\bar{\mu}_t))$

6.  &nbsp;$ \Sigma_t = (I − K_t H_t)\bar{\Sigma}_t$

7.  &nbsp;$ return (\mu_t, \Sigma_t)$


### Predição:

| Descrição                       | Representação no pseudocódigo                     |
|---------------------------------|---------------------------------------------------|
| Estimativa do estado previsto   | $\bar{\mu} _t = g(u_t, \mu_{t-1}$            |
| Covariância prevista do erro    | $\bar{\Sigma}  _t = G_t \Sigma_{t−1} G^T_t + R_t$ |

### Atualização:

| Descrição                             | Representação no pseudocódigo                                      |
|---------------------------------------|--------------------------------------------------------------------|
| Medição residual                      | $z_t − h(\bar{\mu}_t)$                                             |
| Ganho de Kalman                       | $K_t = \bar{\Sigma}_t H^T_t (H_t \bar{\Sigma_t} H^T_t + Q_t)^{−1}$ |
| Atualização da estimativa de estado   | $\mu_t = \bar{\mu}_t + K_t(z_t − h(\bar{\mu}_t))$                  |
| Atualização da covariância            | $\Sigma_t = (I − K_t H_t)\bar{\Sigma}_t$                           |

## Representação das Variáveis

# Explicação do Código

In [6]:
import numpy as np

Iniciamos o robô na posição $[0, 0, 0]^{-1}$, em seguida iniciamos nosso código definindo algumas variáveis importantes:

### Variáveis

**last_position** = Representa a ultima posição do robô $\begin{pmatrix} x_t\\ y_t\\ \psi_t \end{pmatrix}$

**total_movement** = Representa o total de moviment absoluto do robô $\begin{pmatrix} |\Delta x|\\ |\Delta y|\\ |\Delta \psi| \end{pmatrix}$

**current_movement** =  Representa o total de moviment do robô $\begin{pmatrix} \Delta x\\ \Delta y\\ \Delta \psi \end{pmatrix}$

**particles** = lista com as particulas

**updated_marker** = lista com os valores das tag ArUco (boleano)

**aruco_ids** = lista com os valores das tag ArUco (inteiro)

**aruco_list** = lista com as ArUcos

### Subscriber

Dois subscriber serão feitos, um para a Odometria do robô e outro para a biblioteca que detecta o ArUco

### Run

O loop principal executa o FastSLAM quando a tag 0 do ArUco for visualizada pelo robô, que será nossa posição inicial **e** quando e o robô fizer o primeiro movimento

In [7]:
KEY_NUMBER = 1024
aruco_ids = [0, 1, 2, 3]
current_movement = [2, 1, 0]
vetor_normalizado = np.linalg.norm(current_movement)

if aruco_ids[0] != KEY_NUMBER and vetor_normalizado > 0:
    print(vetor_normalizado)

2.23606797749979


Vamos pegar o primeir vetor de movimento do robô e depois resetar esse vetor

In [9]:
def get_movement():
    global current_movement 
    msg = current_movement
    current_movement = np.array([0, 0, 0], dtype='float64').transpose()
    return msg

get_movement()

[2, 1, 0]

Para cada partícula é executada a mesma operação:

1. Adiciona o movimento do robô na particula
2. Percorre a lista aruco_ids:
    1. atualiza a lista updated_marker
    2. Pega a distância e orientação da tag


### Distancia e Orientação da Tag

![Robot Observing](robot_observing.png)

$g(x_t, \theta_{nt}) = \begin{bmatrix} r(s_t, \theta_{nt}) \\\phi(s_t, \theta_{nt}) \end{bmatrix}$ = $\begin{bmatrix} \sqrt{(\theta_{nt}, x - s_t,x)^2 + (\theta_{nt},y - s_t,y)^2} \\ tan^{-1}(\frac{\theta_{nt},y - s_t,y}{\theta_{nt},x - s_t,x} - s_t,\theta) \end{bmatrix}$

# Referências

1. THRUN, Sebastian. Probabilistic robotics. **Communications of the ACM**, v. 45, n. 3, p. 52-57, 2002.
2. MONTEMERLO, Michael et al. FastSLAM: A factored solution to the simultaneous localization and mapping problem. **Aaai/iaai**, v. 593598, 2002.